In [4]:
# aws 계정 로그인 셀
import pymysql
import pandas as pd # 불요. 출력값 보기용.

# 계정 정보 파일에서 필요 인자 딕셔너리로 생성
with open('./secret_db.config') as f:
    res = dict(map(lambda x: x.replace('\n','').split('='), f.readlines()))
    for i, v in res.items():
        if v.isdigit(): res[i] = int(v)
# 로그인 위한 인자명 출력
print(res.keys())
# 딕셔너리 언패킹하여 인자 값 할당 후 서버 연동
remote = pymysql.connect(**res)

dict_keys(['host', 'user', 'password', 'database', 'charset'])


In [2]:
# db 및 유저 생성

# with로 여는 것은 cursor가 여러 개 열리는 것을 예방하기 위함.
with remote.cursor() as cur:
    # db명 설정 및 생성
    dbname = 'db4_6'
    cur.execute(f'create database {dbname}')
    cur.execute('show databases')
    # fetchall(): 출력문 전체 가져오기
    # print(cur.fetchall())
    print(f'{"  show databases  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))
    
    # user 생성 및 권한 지정
    username = 'tmp_01'
    password = 'tmp_1234'   # 연습용이 아닌 실제 생성 user는 secret 파일을 통해 인자 입력 필요
    cur.execute('use mysql')
    # user 생성, %는 외부 접속 사용자라는 의미.
    cur.execute(f'create user \'{username}\'@\'%\' identified by \'{password}\'')
    # 특정 db 권한 전부(*) 부여
    cur.execute(f'grant all on {dbname}.* to \'{username}\'@\'%\'')
    # 특정 user 권한 확인
    cur.execute(f'show grants for \'{username}\'@\'%\'')
    # print(cur.fetchall())
    print(f'''{"  show grants for '{username}'@'%'  ":=^80}''')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['grant users']))

===============================  show databases  ===============================
        database_name
0               db4_6
1  information_schema
2              innodb
3               mysql
4  performance_schema
5                 sys
======================  show grants for '{username}'@'%'  ======================
                                         grant users
0  GRANT USAGE ON *.* TO `tmp_01`@`%` IDENTIFIED ...
1  GRANT ALL PRIVILEGES ON `db4_6`.* TO `tmp_01`@`%`


In [3]:
# db 및 유저 삭제

with remote.cursor() as cur:
    # user 권한 제거 및 user 삭제
    username = 'tmp_01'
    cur.execute('use mysql')
    # user 테이블의 host, user 컬럼 조회
    cur.execute('select host, user from user')
    # print(cur.fetchall())
    print(f'{"  select host, user from user  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['host', 'user']))
    
    # 특정 db 권한 전부(*) 제거
    cur.execute(f'revoke all on {dbname}.* from \'{username}\'@\'%\'')
    # 특정 user 권한 확인
    cur.execute(f'show grants for \'{username}\'@\'%\'')
    # print(cur.fetchall())
    print(f'''{"  show grants for '{username}'@'%'  ":=^80}''')
    print(pd.DataFrame([_ for _ in cur.fetchall()]))

    # 특정 user 삭제
    cur.execute(f'drop user \'{username}\'@\'%\'')
    # user 제거 확인
    cur.execute('select host, user from user')
    # print(cur.fetchall())
    print(f'{"  select host, user from user  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['host', 'user']))


    # db 삭제
    cur.execute('show databases')
    # print(cur.fetchall())
    print(f'{"  show databases  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))
    
    dbname = 'db4_6'
    cur.execute(f'drop database {dbname}')
    # db 삭제 확인
    cur.execute('show databases')
    # print(cur.fetchall())
    print(f'{"  show databases  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))

========================  select host, user from user  =========================
        host         user
0          %         root
1          %       tmp_01
2  localhost  mariadb.sys
3  localhost     rdsadmin
======================  show grants for '{username}'@'%'  ======================
                                                   0
0  GRANT USAGE ON *.* TO `tmp_01`@`%` IDENTIFIED ...
========================  select host, user from user  =========================
        host         user
0          %         root
1  localhost  mariadb.sys
2  localhost     rdsadmin
===============================  show databases  ===============================
        database_name
0               db4_6
1  information_schema
2              innodb
3               mysql
4  performance_schema
5                 sys
===============================  show databases  ===============================
        database_name
0  information_schema
1              innodb
2               mysql
3  performanc